Pair Programming ETL Transformación I


In [2]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
pd.options.display.max_columns=None


El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

Cargaremos los dos ficheros de datos

In [3]:

df= pd.read_csv('../Datos/attacks_limpieza_completa.csv', index_col= 0)
df_clima= pd.read_csv('../Datos/clima_paises.csv')


Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
USA
Australia
New Zealand
South Africa
Papua New Guinea

In [4]:

df_paises = df[df.country.isin(['usa', 'australia', 'new zealand', 'south africa', 'papua new guinea'])]


In [5]:
#segundo metodo
df_paises = df[(df["country"]== "usa") | (df["country"] == "australia") | (df["country"] == "new zealand") | (df["country"] == "south africa") | (df["country"]=="papua new guinea")]
df_paises.country.unique()
df_paises.sample(2)


array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
1613,2004,Unprovoked,usa,27.0,White shark,Oct,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,4,12,7,6.313925
5239,1917,Provoked,usa,27.0,Unespecific,Sep,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,3,13,4,6.313925


Del dataframe de los datos climáticos seleccionaremos todas las columnas.

In [6]:
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 125, 'speed':...",29,-1,9,-9999,none,0,0,125,3,Papua New Guinea
1,6,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 130, 'speed':...",28,-1,10,-9999,none,1,0,135,4,Papua New Guinea


Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

    Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

    Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. 


In [7]:


df_clima['rh_profile'][0]
df_clima['wind_profile'][0]


"[{'layer': '950mb', 'rh': 11}, {'layer': '900mb', 'rh': 15}, {'layer': '850mb', 'rh': 9}, {'layer': '800mb', 'rh': 1}, {'layer': '750mb', 'rh': 1}, {'layer': '700mb', 'rh': 6}, {'layer': '650mb', 'rh': 6}, {'layer': '600mb', 'rh': 11}, {'layer': '550mb', 'rh': 9}, {'layer': '500mb', 'rh': 7}, {'layer': '450mb', 'rh': -2}, {'layer': '400mb', 'rh': -4}, {'layer': '350mb', 'rh': 0}, {'layer': '300mb', 'rh': -1}, {'layer': '250mb', 'rh': 3}, {'layer': '200mb', 'rh': 8}]"

"[{'layer': '950mb', 'direction': 125, 'speed': 4}, {'layer': '900mb', 'direction': 120, 'speed': 4}, {'layer': '850mb', 'direction': 110, 'speed': 5}, {'layer': '800mb', 'direction': 110, 'speed': 4}, {'layer': '750mb', 'direction': 105, 'speed': 4}, {'layer': '700mb', 'direction': 100, 'speed': 4}, {'layer': '650mb', 'direction': 90, 'speed': 4}, {'layer': '600mb', 'direction': 85, 'speed': 4}, {'layer': '550mb', 'direction': 70, 'speed': 4}, {'layer': '500mb', 'direction': 70, 'speed': 4}, {'layer': '450mb', 'direction': 90, 'speed': 4}, {'layer': '400mb', 'direction': 105, 'speed': 3}, {'layer': '350mb', 'direction': 65, 'speed': 3}, {'layer': '300mb', 'direction': 85, 'speed': 4}, {'layer': '250mb', 'direction': 90, 'speed': 5}, {'layer': '200mb', 'direction': 60, 'speed': 5}]"

In [8]:
# os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.
# El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos,
#  es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 

df_clima.dtypes

timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object
temp2m                int64
lifted_index          int64
rh2m                  int64
msl_pressure          int64
prec_type            object
prec_amount           int64
snow_depth            int64
wind10m.direction     int64
wind10m.speed         int64
country              object
dtype: object

In [9]:

# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su 
# tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos
#  de la siguiente forma: 

import ast

df_clima['wind_profile']= df_clima['wind_profile'].apply(ast.literal_eval)

df_clima['rh_profile']= df_clima['rh_profile'].apply(ast.literal_eval)


In [10]:


# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores 
# son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna 
# será key del diccionario o cada elemento de la lista. 
# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 

x = df_clima['rh_profile'].apply(pd.Series) 
x[0].apply(pd.Series)['layer'][0]
x
# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta 
# compuesta por distintos diccionarios)


'950mb'

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 11}","{'layer': '900mb', 'rh': 15}","{'layer': '850mb', 'rh': 9}","{'layer': '800mb', 'rh': 1}","{'layer': '750mb', 'rh': 1}","{'layer': '700mb', 'rh': 6}","{'layer': '650mb', 'rh': 6}","{'layer': '600mb', 'rh': 11}","{'layer': '550mb', 'rh': 9}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': -4}","{'layer': '350mb', 'rh': 0}","{'layer': '300mb', 'rh': -1}","{'layer': '250mb', 'rh': 3}","{'layer': '200mb', 'rh': 8}"
1,"{'layer': '950mb', 'rh': 13}","{'layer': '900mb', 'rh': 15}","{'layer': '850mb', 'rh': 13}","{'layer': '800mb', 'rh': 11}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 3}","{'layer': '650mb', 'rh': 6}","{'layer': '600mb', 'rh': 8}","{'layer': '550mb', 'rh': 8}","{'layer': '500mb', 'rh': 5}","{'layer': '450mb', 'rh': 5}","{'layer': '400mb', 'rh': -4}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 0}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': -1}"
2,"{'layer': '950mb', 'rh': 12}","{'layer': '900mb', 'rh': 14}","{'layer': '850mb', 'rh': 4}","{'layer': '800mb', 'rh': -2}","{'layer': '750mb', 'rh': -1}","{'layer': '700mb', 'rh': 1}","{'layer': '650mb', 'rh': 5}","{'layer': '600mb', 'rh': 10}","{'layer': '550mb', 'rh': 8}","{'layer': '500mb', 'rh': 2}","{'layer': '450mb', 'rh': 6}","{'layer': '400mb', 'rh': -2}","{'layer': '350mb', 'rh': -2}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': 1}"
3,"{'layer': '950mb', 'rh': 13}","{'layer': '900mb', 'rh': 15}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': -2}","{'layer': '750mb', 'rh': -2}","{'layer': '700mb', 'rh': 0}","{'layer': '650mb', 'rh': 3}","{'layer': '600mb', 'rh': 9}","{'layer': '550mb', 'rh': 8}","{'layer': '500mb', 'rh': 5}","{'layer': '450mb', 'rh': 4}","{'layer': '400mb', 'rh': 0}","{'layer': '350mb', 'rh': -2}","{'layer': '300mb', 'rh': 2}","{'layer': '250mb', 'rh': 1}","{'layer': '200mb', 'rh': 0}"
4,"{'layer': '950mb', 'rh': 12}","{'layer': '900mb', 'rh': 14}","{'layer': '850mb', 'rh': 2}","{'layer': '800mb', 'rh': -3}","{'layer': '750mb', 'rh': -3}","{'layer': '700mb', 'rh': -1}","{'layer': '650mb', 'rh': 2}","{'layer': '600mb', 'rh': 4}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': 4}","{'layer': '400mb', 'rh': 4}","{'layer': '350mb', 'rh': -1}","{'layer': '300mb', 'rh': 4}","{'layer': '250mb', 'rh': 0}","{'layer': '200mb', 'rh': -1}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,"{'layer': '950mb', 'rh': 13}","{'layer': '900mb', 'rh': 13}","{'layer': '850mb', 'rh': 15}","{'layer': '800mb', 'rh': 14}","{'layer': '750mb', 'rh': -1}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': 6}","{'layer': '600mb', 'rh': 12}","{'layer': '550mb', 'rh': 10}","{'layer': '500mb', 'rh': 13}","{'layer': '450mb', 'rh': 16}","{'layer': '400mb', 'rh': 16}","{'layer': '350mb', 'rh': 13}","{'layer': '300mb', 'rh': 15}","{'layer': '250mb', 'rh': 16}","{'layer': '200mb', 'rh': -3}"
316,"{'layer': '950mb', 'rh': 11}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 13}","{'layer': '800mb', 'rh': -3}","{'layer': '750mb', 'rh': -4}","{'layer': '700mb', 'rh': -4}","{'layer': '650mb', 'rh': -3}","{'layer': '600mb', 'rh': -4}","{'layer': '550mb', 'rh': 2}","{'layer': '500mb', 'rh': 12}","{'layer': '450mb', 'rh': 10}","{'layer': '400mb', 'rh': 16}","{'layer': '350mb', 'rh': 16}","{'layer': '300mb', 'rh': 16}","{'layer': '250mb', 'rh': 16}","{'layer': '200mb', 'rh': -3}"
317,"{'layer': '950mb', 'rh': 11}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 14}","{'layer': '800mb', 'rh': -1}","{'layer': '750mb', 'rh': -2}","{'layer': '700mb', 'rh': -2}","{'layer': '650mb', 'rh': -2}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': -2}","{'layer': '500mb', 'rh': 11}","{'layer': '450mb', 'rh': 15}","{'layer': '400mb', 'rh': 16}","{'layer': '350mb', 'rh': 16}","{'layer': '300mb', 'rh': 16}","{'layer': '25

In [11]:
# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los 
# diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los 
# values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 

# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)['layer'][0]) 

    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df_clima.insert(i, nombre, valores)


In [12]:

df_clima.columns


Index(['rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb',
       'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb',
       'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb', 'timepoint',
       'cloudcover', 'highcloud', 'midcloud', 'lowcloud', 'rh_profile',
       'wind_profile', 'temp2m', 'lifted_index', 'rh2m', 'msl_pressure',
       'prec_type', 'prec_amount', 'snow_depth', 'wind10m.direction',
       'wind10m.speed', 'country'],
      dtype='object')

In [13]:
df_clima.head()

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,11,15,9,1,1,6,6,11,9,7,-2,-4,0,-1,3,8,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 125, 'speed':...",29,-1,9,-9999,none,0,0,125,3,Papua New Guinea
1,13,15,13,11,3,3,6,8,8,5,5,-4,-1,0,2,-1,6,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 130, 'speed':...",28,-1,10,-9999,none,1,0,135,4,Papua New Guinea
2,12,14,4,-2,-1,1,5,10,8,2,6,-2,-2,2,2,1,9,8,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 120, 'speed':...",29,-1,10,-9999,none,1,0,120,4,Papua New Guinea
3,13,15,10,-2,-2,0,3,9,8,5,4,0,-2,2,1,0,12,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 110, 'speed':...",29,-4,10,-9999,none,1,0,105,4,Papua New Guinea
4,12,14,2,-3,-3,-1,2,4,5,7,4,4,-1,4,0,-1,15,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 105, 'speed':...",29,-4,10,-9999,none,1,0,110,3,Papua New Guinea


In [14]:
# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe 
# donde cada fila será una celda del dataframe anterior. 


In [15]:
#hacenos lo mismo para la columna wind
y = df_clima['wind_profile'].apply(pd.Series) 
y[0].apply(pd.Series)['layer'][0]
y


'950mb'

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 125, 'speed': 4}","{'layer': '900mb', 'direction': 120, 'speed': 4}","{'layer': '850mb', 'direction': 110, 'speed': 5}","{'layer': '800mb', 'direction': 110, 'speed': 4}","{'layer': '750mb', 'direction': 105, 'speed': 4}","{'layer': '700mb', 'direction': 100, 'speed': 4}","{'layer': '650mb', 'direction': 90, 'speed': 4}","{'layer': '600mb', 'direction': 85, 'speed': 4}","{'layer': '550mb', 'direction': 70, 'speed': 4}","{'layer': '500mb', 'direction': 70, 'speed': 4}","{'layer': '450mb', 'direction': 90, 'speed': 4}","{'layer': '400mb', 'direction': 105, 'speed': 3}","{'layer': '350mb', 'direction': 65, 'speed': 3}","{'layer': '300mb', 'direction': 85, 'speed': 4}","{'layer': '250mb', 'direction': 90, 'speed': 5}","{'layer': '200mb', 'direction': 60, 'speed': 5}"
1,"{'layer': '950mb', 'direction': 130, 'speed': 4}","{'layer': '900mb', 'direction': 125, 'speed': 4}","{'layer': '850mb', 'direction': 115, 'speed': 4}","{'layer': '800mb', 'direction': 105, 'speed': 4}","{'layer': '750mb', 'direction': 100, 'speed': 4}","{'layer': '700mb', 'direction': 95, 'speed': 4}","{'layer': '650mb', 'direction': 90, 'speed': 4}","{'layer': '600mb', 'direction': 80, 'speed': 4}","{'layer': '550mb', 'direction': 75, 'speed': 4}","{'layer': '500mb', 'direction': 85, 'speed': 3}","{'layer': '450mb', 'direction': 75, 'speed': 3}","{'layer': '400mb', 'direction': 110, 'speed': 3}","{'layer': '350mb', 'direction': 70, 'speed': 3}","{'layer': '300mb', 'direction': 65, 'speed': 5}","{'layer': '250mb', 'direction': 80, 'speed': 5}","{'layer': '200mb', 'direction': 85, 'speed': 5}"
2,"{'layer': '950mb', 'direction': 120, 'speed': 4}","{'layer': '900mb', 'direction': 115, 'speed': 4}","{'layer': '850mb', 'direction': 110, 'speed': 4}","{'layer': '800mb', 'direction': 110, 'speed': 4}","{'layer': '750mb', 'direction': 105, 'speed': 4}","{'layer': '700mb', 'direction': 95, 'speed': 4}","{'layer': '650mb', 'direction': 90, 'speed': 5}","{'layer': '600mb', 'direction': 85, 'speed': 5}","{'layer': '550mb', 'direction': 85, 'speed': 4}","{'layer': '500mb', 'direction': 80, 'speed': 3}","{'layer': '450mb', 'direction': 65, 'speed': 3}","{'layer': '400mb', 'direction': 110, 'speed': 3}","{'layer': '350mb', 'direction': 80, 'speed': 3}","{'layer': '300mb', 'direction': 65, 'speed': 5}","{'layer': '250mb', 'direction': 75, 'speed': 5}","{'layer': '200mb', 'direction': 80, 'speed': 5}"
3,"{'layer': '950mb', 'direction': 110, 'speed': 4}","{'layer': '900mb', 'direction': 110, 'speed': 4}","{'layer': '850mb', 'direction': 110, 'speed': 4}","{'layer': '800mb', 'direction': 115, 'speed': 4}","{'layer': '750mb', 'direction': 105, 'speed': 4}","{'layer': '700mb', 'direction': 100, 'speed': 4}","{'layer': '650mb', 'direction': 95, 'speed': 5}","{'layer': '600mb', 'direction': 95, 'speed': 5}","{'layer': '550mb', 'direction': 90, 'speed': 4}","{'layer': '500mb', 'direction': 80, 'speed': 3}","{'layer': '450mb', 'direction': 115, 'speed': 2}","{'layer': '400mb', 'direction': 85, 'speed': 3}","{'layer': '350mb', 'direction': 75, 'speed': 4}","{'layer': '300mb', 'direction': 75, 'speed': 5}","{'layer': '250mb', 'direction': 65, 'speed': 5}","{'layer': '200mb', 'direction': 60, 'speed': 5}"
4,"{'layer': '950mb', 'direction': 105, 'speed': 4}","{'layer': '900mb', 'direction': 110, 'speed': 4}","{'layer': '850mb', 'direction': 120, 'speed': 4}","{'layer': '800mb', 'direction': 120, 'speed': 4}","{'layer': '750mb', 'direction': 115, 'speed': 4}","{'layer': '700mb', 'direction': 105, 'speed': 4}","{'layer': '650mb', 'direction': 100, 'speed': 4}","{'layer': '600mb', 'direction': 100, 'speed': 4}","{'layer': '550mb', 'direction': 100, 'speed': 4}","{'layer': '500mb', 'direction': 80, 'speed': 3}","{'layer': '450mb', 'direction': 55, 'speed': 3}","{'layer': '400mb', 'direction': 70, 'speed': 3}","{'layer': '350mb', 'direction': 90, 'speed': 5}","{'layer': '300mb', 'direction': 80, 'speed': 5}","{'layer': '250m

In [16]:
for i in range(len(y.columns)): 

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "wi_speed" + str(y[i].apply(pd.Series)['layer'][0]) 
    valores = list(y[i].apply(pd.Series)["speed"])
    df_clima.insert(i, nombre, valores)
    
    nombre2 = "wi_direction" + str(y[i].apply(pd.Series)['layer'][0]) 
    valores2 = list(y[i].apply(pd.Series)["direction"])
    df_clima.insert(i, nombre2, valores2)


In [17]:
df_clima.columns
df_clima.shape


Index(['wi_direction950mb', 'wi_direction900mb', 'wi_direction850mb',
       'wi_direction800mb', 'wi_direction750mb', 'wi_direction700mb',
       'wi_direction650mb', 'wi_direction600mb', 'wi_direction550mb',
       'wi_direction500mb', 'wi_direction450mb', 'wi_direction400mb',
       'wi_direction350mb', 'wi_direction300mb', 'wi_direction250mb',
       'wi_direction200mb', 'wi_speed200mb', 'wi_speed250mb', 'wi_speed300mb',
       'wi_speed350mb', 'wi_speed400mb', 'wi_speed450mb', 'wi_speed500mb',
       'wi_speed550mb', 'wi_speed600mb', 'wi_speed650mb', 'wi_speed700mb',
       'wi_speed750mb', 'wi_speed800mb', 'wi_speed850mb', 'wi_speed900mb',
       'wi_speed950mb', 'rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb',
       'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb',
       'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb',
       'timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'rh_profile', 'wind_profile', 'temp2m'

(320, 65)

In [18]:
df_clima.head(2)

,wi_direction950mb,wi_direction900mb,wi_direction850mb,wi_direction800mb,wi_direction750mb,wi_direction700mb,wi_direction650mb,wi_direction600mb,wi_direction550mb,wi_direction500mb,wi_direction450mb,wi_direction400mb,wi_direction350mb,wi_direction300mb,wi_direction250mb,wi_direction200mb,wi_speed200mb,wi_speed250mb,wi_speed300mb,wi_speed350mb,wi_speed400mb,wi_speed450mb,wi_speed500mb,wi_speed550mb,wi_speed600mb,wi_speed650mb,wi_speed700mb,wi_speed750mb,wi_speed800mb,wi_speed850mb,wi_speed900mb,wi_speed950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,125,120,110,110,105,100,90,85,70,70,90,105,65,85,90,60,5,5,4,3,3,4,4,4,4,4,4,4,4,5,4,4,11,15,9,1,1,6,6,11,9,7,-2,-4,0,-1,3,8,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 125, 'speed':...",29,-1,9,-9999,none,0,0,125,3,Papua New Guinea
1,130,125,115,105,100,95,90,80,75,85,75,110,70,65,80,85,5,5,5,3,3,3,3,4,4,4,4,4,4,4,4,4,13,15,13,11,3,3,6,8,8,5,5,-4,-1,0,2,-1,6,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 130, 'speed':...",28,-1,10,-9999,none,1,0,135,4,Papua New Guinea


In [19]:
df_clima.to_csv('../Datos/datos_clima.csv', index=False)
df_paises.to_csv('../Datos/datos_paises.csv', index=False)

In [20]:

# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 

df_mean = df_clima.groupby("country").mean()
df_mean

,wi_direction950mb,wi_direction900mb,wi_direction850mb,wi_direction800mb,wi_direction750mb,wi_direction700mb,wi_direction650mb,wi_direction600mb,wi_direction550mb,wi_direction500mb,wi_direction450mb,wi_direction400mb,wi_direction350mb,wi_direction300mb,wi_direction250mb,wi_direction200mb,wi_speed200mb,wi_speed250mb,wi_speed300mb,wi_speed350mb,wi_speed400mb,wi_speed450mb,wi_speed500mb,wi_speed550mb,wi_speed600mb,wi_speed650mb,wi_speed700mb,wi_speed750mb,wi_speed800mb,wi_speed850mb,wi_speed900mb,wi_speed950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
New Zealand,93.906250,96.015625,95.468750,92.734375,92.187500,92.812500,92.578125,89.140625,84.531250,79.531250,130.859375,104.921875,135.703125,148.750000,184.765625,227.421875,3.843750,3.062500,3.078125,2.343750,2.484375,2.609375,3.000000,3.546875,3.765625,3.781250,3.890625,3.921875,3.953125,3.812500,3.484375,3.234375,12.687500,12.515625,5.531250,3.328125,3.328125,3.593750,3.046875,3.546875,3.140625,1.234375,0.156250,-1.171875,-0.015625,1.625000,2.312500,3.875000,97.5,3.109375,-9999.0,-9999.0,-9999.0,27.250000,-0.390625,10.234375,-9999.000000,1.656250,0.0,95.156250,3.031250
Papua New Guinea,92.500000,96.015625,99.609375,100.703125,99.921875,98.125000,96.484375,95.781250,94.687500,90.390625,94.765625,102.890625,103.203125,101.328125,84.296875,82.656250,5.265625,4.750000,4.656250,4.468750,4.234375,4.281250,4.203125,4.390625,4.187500,4.125000,3.937500,3.843750,3.562500,3.406250,3.234375,2.984375,12.062500,13.171875,8.234375,4.796875,4.093750,4.359375,5.031250,6.343750,6.140625,4.562500,3.781250,2.671875,2.390625,1.578125,0.828125,5.609375,97.5,6.468750,-9999.0,-9999.0,-9999.0,28.515625,-2.125000,9.578125,-9999.000000,2.437500,0.0,92.812500,2.781250
South Africa,230.000000,204.843750,201.093750,147.500000,93.046875,103.359375,107.031250,102.343750,93.359375,88.593750,90.625000,103.828125,106.171875,94.531250,89.609375,73.671875,4.687500,3.437500,3.140625,3.375000,3.421875,3.234375,3.109375,3.078125,3.250000,3.296875,2.906250,2.468750,2.250000,2.390625,2.531250,2.109375,12.656250,10.812500,10.468750,10.640625,11.062500,11.328125,12.343750,13.546875,13.828125,12.515625,11.546875,10.515625,9.765625,10.093750,11.234375,12.406250,97.5,8.421875,-9999.0,-9999.0,-9999.0,22.093750,-1.078125,13.375000,-9999.000000,4.515625,0.0,232.500000,1.921875
australia,103.671875,103.750000,106.875000,106.796875,104.843750,101.250000,98.281250,94.609375,92.421875,93.359375,93.046875,102.031250,102.109375,90.078125,73.515625,64.843750,6.281250,5.812500,5.265625,4.953125,4.968750,4.609375,4.546875,4.484375,4.328125,4.125000,3.890625,3.734375,3.359375,3.156250,2.750000,2.453125,12.796875,11.890625,10.000000,8.468750,7.281250,7.234375,7.468750,7.953125,7.640625,6.343750,6.078125,5.343750,5.609375,4.218750,6.828125,7.156250,97.5,6.656250,-9999.0,-9999.0,-9999.0,28.125000,-2.593750,9.750000,-9999.000000,3.265625,0.0,108.359375,2.312500
usa,255.000000,260.078125,260.312500,260.859375,261.484375,261.640625,262.265625,261.250000,261.328125,261.484375,259.765625,259.921875,259.765625,240.390625,246.562500,260.078125,7.531250,7.828125,7.812500,7.375000,7.171875,6.671875,6.140625,5.640625,5.250000,5.109375,4.921875,4.765625,4.625000,4.453125,4.406250,4.312500,12.031250,12.281250,13.296875,10.750000,7.031250,5.828125,5.671875,5.062500,5.265625,5.593750,5.218750,5.562500,6.125000,5.859375,5.437500,-0.078125,97.5,8.375000,-9999.0,-9999.0,-9999.0,9.562500,9.140625,10.390625,1022.359375,3.281250,0.0,200.312500,3.765625


In [21]:

df_final = pd.merge(left = df_paises, right= df_mean, how= "left", left_on= "country", right_on= "country")
df_final


,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,wi_direction950mb,wi_direction900mb,wi_direction850mb,wi_direction800mb,wi_direction750mb,wi_direction700mb,wi_direction650mb,wi_direction600mb,wi_direction550mb,wi_direction500mb,wi_direction450mb,wi_direction400mb,wi_direction350mb,wi_direction300mb,wi_direction250mb,wi_direction200mb,wi_speed200mb,wi_speed250mb,wi_speed300mb,wi_speed350mb,wi_speed400mb,wi_speed450mb,wi_speed500mb,wi_speed550mb,wi_speed600mb,wi_speed650mb,wi_speed700mb,wi_speed750mb,wi_speed800mb,wi_speed850mb,wi_speed900mb,wi_speed950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,255.000000,260.078125,260.3125,260.859375,261.484375,261.640625,262.265625,261.250000,261.328125,261.484375,259.765625,259.921875,259.765625,240.390625,246.562500,260.078125,7.53125,7.828125,7.812500,7.375000,7.171875,6.671875,6.140625,5.640625,5.250000,5.109375,4.921875,4.765625,4.625000,4.453125,4.40625,4.312500,12.031250,12.281250,13.296875,10.75000,7.03125,5.828125,5.671875,5.062500,5.265625,5.59375,5.218750,5.56250,6.125000,5.859375,5.437500,-0.078125,97.5,8.37500,-9999.0,-9999.0,-9999.0,9.5625,9.140625,10.390625,1022.359375,3.281250,0.0,200.312500,3.765625
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,255.000000,260.078125,260.3125,260.859375,261.484375,261.640625,262.265625,261.250000,261.328125,261.484375,259.765625,259.921875,259.765625,240.390625,246.562500,260.078125,7.53125,7.828125,7.812500,7.375000,7.171875,6.671875,6.140625,5.640625,5.250000,5.109375,4.921875,4.765625,4.625000,4.453125,4.40625,4.312500,12.031250,12.281250,13.296875,10.75000,7.03125,5.828125,5.671875,5.062500,5.265625,5.59375,5.218750,5.56250,6.125000,5.859375,5.437500,-0.078125,97.5,8.37500,-9999.0,-9999.0,-9999.0,9.5625,9.140625,10.390625,1022.359375,3.281250,0.0,200.312500,3.765625
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629,255.000000,260.078125,260.3125,260.859375,261.484375,261.640625,262.265625,261.250000,261.328125,261.484375,259.765625,259.921875,259.765625,240.390625,246.562500,260.078125,7.53125,7.828125,7.812500,7.375000,7.171875,6.671875,6.140625,5.640625,5.250000,5.109375,4.921875,4.765625,4.625000,4.453125,4.40625,4.312500,12.031250,12.281250,13.296875,10.75000,7.03125,5.828125,5.671875,5.062500,5.265625,5.59375,5.218750,5.56250,6.125000,5.859375,5.437500,-0.078125,97.5,8.37500,-9999.0,-9999.0,-9999.0,9.5625,9.140625,10.390625,1022.359375,3.281250,0.0,200.312500,3.765625
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925,103.671875,103.750000,106.8750,106.796875,104.843750,101.250000,98.281250,94.609375,92.421875,93.359375,93.046875,102.031250,102.109375,90.078125,73.515625,64.843750,6.28125,5.812500,5.265625,4.953125,4.968750,4.609375,4.546875,4.484375,4.328125,4.125000,3.890625,3.734375,3.359375,3.156250,2.75000,2.453125,12.796875,11.890625,10.000000,8.46875,7.28125,7.234375,7.468750,7.953125,7.640625,6.34375,6.078125,5.34375,5.609375,4.218750,6.828125,7.156250,97.5,6.65625,-9999.0,-9999.0,-9999.0,28.1250,-2.593750,9.750000,-9999.000000,3.265625,0.0,108.359375,2.312500
4,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925,103.671875,103.750000,106.8750,106.796875,104.843750,101.250000,98.281250,94.609375,92.421875,93.359375,93.046875,102.031250,102.109375,90.078125,73.515625,64.843750,6.28125,5.812500,5.265625,4.953125,4.968750,4.609375,4.546875,4.484375,4.32812

Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.
Happy coding 🦈

In [22]:
df_final.to_csv('../Datos/etl_transformacion.csv', index=False)